In [1]:
import deepchem as dc
import torch


No normalization for AvgIpc. Feature removed!
Skipped loading some Jax models, missing a dependency. No module named 'jax'


In [2]:
tasks, datasets, transformers = dc.molnet.load_delaney(
    featurizer="ECFP", splitter="random"
)
train_dataset, valid_dataset, test_dataset = datasets
metric = dc.metrics.Metric(dc.metrics.pearson_r2_score)

In [5]:
pytorch_model = torch.nn.Sequential(
    torch.nn.Linear(1024, 1000),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.5),
    torch.nn.Linear(1000, 1),
)
model = dc.models.TorchModel(pytorch_model, dc.models.losses.L2Loss())

model.fit(train_dataset, nb_epoch=50)
print("training set score:", model.evaluate(train_dataset, [metric]))
print("test set score:", model.evaluate(test_dataset, [metric]))

training set score: {'pearson_r2_score': 0.9765674560919542}
test set score: {'pearson_r2_score': 0.6783649748785865}


In [6]:
class ClassificationModel(torch.nn.Module):
    def __init__(self):
        super(ClassificationModel, self).__init__()
        self.dense1 = torch.nn.Linear(1024, 1000)
        self.dense2 = torch.nn.Linear(1000, 1)

    def forward(self, inputs):
        y = torch.nn.functional.relu(self.dense1(inputs))
        y = torch.nn.functional.dropout(y, p=0.5, training=self.training)
        logits = self.dense2(y)
        output = torch.sigmoid(logits)
        return output, logits

In [7]:
torch_model = ClassificationModel()
output_types = ["prediction", "loss"]
model = dc.models.TorchModel(
    torch_model, dc.models.losses.SigmoidCrossEntropy(), output_types=output_types
)

In [8]:
tasks, datasets, transformers = dc.molnet.load_bace_classification(
    feturizer="ECFP", splitter="scaffold"
)
train_dataset, valid_dataset, test_dataset = datasets
model.fit(train_dataset, nb_epoch=100)
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
print("training set score:", model.evaluate(train_dataset, [metric]))
print("test set score:", model.evaluate(test_dataset, [metric]))

training set score: {'roc_auc_score': 0.9996367954180345}
test set score: {'roc_auc_score': 0.7698822463768116}
